In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

#### Reading in Audio Features

In [2]:
Compare = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
track_elems = pd.read_pickle('../Pickle_Files/track_elems.pkl')

#### Drops all uncommon songs

In [3]:
Compare = pd.merge(audio_features[['track_uri']],Compare,on='track_uri')

In [4]:
popularity_ratings = pd.read_pickle('../Pickle_Files/Song_Popularity.pkl')

In [5]:
Compare = pd.merge(Compare,popularity_ratings[['Popularity_Rating_Normalized','track_uri']],on='track_uri')

#### Ordering tracks

In [6]:
Compare = Compare.sort_values('track_uri',ascending=True)

In [7]:
Compare = Compare.reset_index().drop(['index'],axis=1)

In [8]:
len(Compare)

2254670

#### Creating Dataframe 'Cosine'

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
cosine = pd.read_pickle('../Pickle_Files/cosine_dataframe.pkl')

In [11]:
X = cosine.values

#### Normalizing values before cosine calculated

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit(cosine.values).transform(cosine.values)

# Song Recommender System
<br>-  This Recommender System takes in an input song dataframe and makes 10 recommendations for the song.

In [13]:
def Song_Recommender_System(df):
    
    dfvalues = df.index.values
    
    num_tracks_desired = 10 ## Recommends ten songs
    def rec_top_n(n):
        return np.argsort(cosine_similarity(X, X[dfvalues]), axis=0)[::-1][0:n] # Cosine similarity finds closest to each song
    # Prints index of all recommended songs
    print(rec_top_n(num_tracks_desired+1))
    test = rec_top_n(num_tracks_desired+1).flatten()

    test = Compare.iloc[test]
    test = test[['track_name','artist_name','track_uri','album_name','Popularity_Rating_Normalized']]
    return test

### Example with Popular DJ's

In [14]:
one = Compare[Compare['artist_name']=='Avicii'].sort_values(
    'Popularity_Rating_Normalized',ascending=False).head(10) 
two = Compare[Compare['artist_name']=='Hardwell'].sort_values(
    'Popularity_Rating_Normalized',ascending=False).head(10)   
three = Compare[Compare['artist_uri']==('spotify:artist:2CIMQHirSU0MQqyYHq0eOx')].sort_values(
    'Popularity_Rating_Normalized',ascending=False).head(10)#deadmau5
four = Compare[Compare['artist_name']=='Swedish House Mafia'].sort_values(
    'Popularity_Rating_Normalized',ascending=False).head(10)   
five = Compare[Compare['artist_name']=='Calvin Harris'].sort_values(
    'Popularity_Rating_Normalized',ascending=False).head(10)  

In [15]:
cols = five.columns

all_songs = pd.concat([one.sample(5), two.sample(5),three.sample(5),four.sample(5),five.sample(5)], ignore_index=False)

### Picks one random song from dataframe to recommend

In [16]:
sample_recommendation = all_songs.sample(1)

In [17]:
Song_Recommender_System(sample_recommendation)

[[ 561631]
 [ 936600]
 [ 812703]
 [1149062]
 [1276587]
 [1560908]
 [  41282]
 [ 355658]
 [2247051]
 [ 741845]
 [1594360]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
561631,My Way,Calvin Harris,spotify:track:1vvNmPOiUuyCbgWmtc6yfm,My Way,0.820719
936600,Strangers Like Me,Phil Collins,spotify:track:3E99KpWAcYEnj64e86y6u9,Tarzan Original Soundtrack,0.759834
812703,Closer,Ne-Yo,spotify:track:2nbClS09zsIAqNkshg6jnp,Year Of The Gentleman,0.738610
1149062,Bodies,Drowning Pool,spotify:track:3xktQXIr1OD3ENc19viwDP,Sinner,0.752300
1276587,Message Man,Twenty One Pilots,spotify:track:4Oyl6oYSNeeZZP0OAxPVaU,Blurryface,0.700683
1560908,UnFazed (feat. The Weeknd),Lil Uzi Vert,spotify:track:5O8x1YlmLQopQHErxtDfi9,Luv Is Rage 2,0.700822
41282,Sway - Chainsmokers Remix,Anna Of The North,spotify:track:08qjPXZBdEyif5PxU2AY9S,Sway,0.716717
355658,Past Lives,BØRNS,spotify:track:1Dr5JexwA15wmKe7Y7maA9,Dopamine,0.720817
2247051,Where Them Girls At (feat. Nicki Minaj & Flo R...,David Guetta,spotify:track:7yO48FWUkqsrdxrwkGcnwl,Nothing But the Beat Ultimate,0.754023
741845,I Run to You,Lady Antebellum,spotify:track:2YVCkTYJpv5jelYrHYoEG8,Lady Antebellum,0.640849


## Widget to give a recommendation for every song
<br> -Simply choose a song using the slider for the top ten recommendations for that song.

In [18]:
import ipywidgets as widgets
from ipywidgets import Layout

style = {'description_width': 'initial'}

choose_song = widgets.IntSlider(
    value=1764009,
    min=0,
    max=Compare.track_uri.nunique(),
    step=1,
    description='Song Index:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(choose_song)

IntSlider(value=1764009, continuous_update=False, description='Song Index:', max=2254670)

In [19]:
random_sample_recommendation = Compare[Compare.index == [choose_song.value]]

#### Save the World by Swedish House Mafia
<br>- The song choosen on the slider is by Swedish House Mafia a group of DJs
<br>- We can see the first recommendation below is the song Clarity by Zedd another DJ this is a great recommendation.
<br>- We can see other recommendations for this song include Avicii, Seven Lions, Martin Garrix and many more very famous DJs

In [20]:
Song_Recommender_System(random_sample_recommendation)

[[1764009]
 [ 474268]
 [1923870]
 [2252132]
 [ 782656]
 [ 885580]
 [1721128]
 [2210830]
 [ 825302]
 [1536882]
 [1311849]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
1764009,Save the World - Radio Mix,Swedish House Mafia,spotify:track:65hRZOQkVjmBdusz1N7aOv,Save The World,0.665777
474268,Spectrum,Zedd,spotify:track:1dFkD1JfRMzwO6hwUsE8aS,Clarity,0.740927
1923870,Fly To The Angels,Slaughter,spotify:track:6e165g8fp2WDCBwF1zxLMh,Mass Slaughter,0.566570
2252132,Light,San Holo,spotify:track:7zTx8ePYAmPFQuxP3xlXZn,Light (Remixes) - EP,0.760978
782656,Divine Sorrow,Wyclef Jean,spotify:track:2h9DxDhkLPpYiEdBtpLykG,Divine Sorrow,0.595251
885580,Sleep Alone,Two Door Cinema Club,spotify:track:33CehYfkie4ugV2gWaTKlM,Beacon,0.650776
1721128,Crystals,Of Monsters and Men,spotify:track:5wU6jk9kxYzFGUpeE6T2Q5,Beneath The Skin,0.730333
2210830,Don't Look Down,Martin Garrix,spotify:track:7qbIEBAzM8PDwtFAwZR3s5,The Martin Garrix Collection,0.740402
825302,Strangers,Seven Lions,spotify:track:2qIWuOrAN2aGyAnh6K5nxe,Strangers,0.737187
1536882,The Days,Avicii,spotify:track:5Iy2Jj87Ha0C0IBlNE1I4y,The Days / Nights,0.692353


# Song Recommendation Examples

## Phantoms Can't Hang - deadmau5

In [21]:
recommend = Compare[Compare['track_uri']=='spotify:track:1Lw2BPiKexowWGfkq0cAkr']

In [22]:
recommend[['track_name','artist_name','track_uri','album_name','Popularity_Rating_Normalized']]

,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
393586,Phantoms Can't Hang,deadmau5,spotify:track:1Lw2BPiKexowWGfkq0cAkr,while(1<2),0.589071


#### Recommendation Analysis
<br>- A very clear observation can be made here that all the returned songs are also from other DJs
<br>- The recommender system has also returned another song my deadmau5 'Pets'
<br>- The recommended DJs are all chill and progressive DJs like Jamie xx and ODESZA which means they are good recommendations

In [23]:
Song_Recommender_System(recommend)

[[ 393586]
 [1438177]
 [1897763]
 [ 333608]
 [2066538]
 [ 828769]
 [1028124]
 [ 989038]
 [ 300628]
 [ 560272]
 [1840387]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
393586,Phantoms Can't Hang,deadmau5,spotify:track:1Lw2BPiKexowWGfkq0cAkr,while(1<2),0.589071
1438177,The Way U Do,Shlohmo,spotify:track:4xl7p1wHcDAihFHudcfn25,Vacation - Single,0.480598
1897763,Sleep Sound,Jamie xx,spotify:track:6YPQGcqfMXZbQaz6l30uBJ,Girl/Sleep Sound,0.567860
333608,Memories That You Call,ODESZA,spotify:track:198dZZVS2hHVMhcQapDFOW,In Return,0.644572
2066538,Pets,deadmau5,spotify:track:78hbC4snQ79I7UcmGi4xxw,while(1<2),0.583863
828769,Love,Daughter,spotify:track:2r28VvWB0kMA3o4epSlcJG,Wild Youth EP,0.475063
1028124,Standing,Hansel Thorn,spotify:track:3XsBi9F4RbD4nJI21019Yd,Hansel Thorn,0.374797
989038,Solo Dancing,Indiana,spotify:track:3PS7vZHBOB3rT70CUYHagZ,No Romeo (Deluxe),0.426298
300628,The Winner Is - from Little Miss Sunshine,DeVotchKa,spotify:track:127dZoLOYo7zA2bXXs8hav,Fox Searchlight Pictures: 20th Anniversary Album,0.473577
560272,Spectre,Tycho,spotify:track:1vdN00jfVssgwJBMwmSAoT,Awake,0.484852


## Take Care - Drake

In [24]:
recommend = Compare[Compare['track_uri']=='spotify:track:124NFj84ppZ5pAxTuVQYCQ']

In [25]:
recommend[['track_name','artist_name','track_uri','album_name','Popularity_Rating_Normalized']]

,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
300409,Take Care,Drake,spotify:track:124NFj84ppZ5pAxTuVQYCQ,Take Care,0.82483


#### Recommendation Analysis
<br>- A very clear observation can be made here that all the returned songs are also from other Rapper like Kendrick Lamar and Childish Gambino
<br>- The recommender system has also returned another song my Drake from the same album 'Make Me Proud', this shows the recommender system can identify an artist with having any information about them
<br>- The recommended songs are all from other popular artists similar to Drake like G-Eazy, Chris Brown and Young Thug

In [26]:
Song_Recommender_System(recommend)

[[ 300409]
 [ 953685]
 [2248352]
 [ 292012]
 [ 504684]
 [1819388]
 [1553751]
 [1162099]
 [2254455]
 [ 387873]
 [ 185571]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
300409,Take Care,Drake,spotify:track:124NFj84ppZ5pAxTuVQYCQ,Take Care,0.824830
953685,Heartbeat,Childish Gambino,spotify:track:3HooZZPp0evFShqaJ2Pwer,Camp,0.795680
2248352,Make Me Proud,Drake,spotify:track:7yfg0Eer6UZZt5tZ1XdsWz,Take Care,0.800857
292012,Really Really,Kevin Gates,spotify:track:10I3CmmwT0BkOVhduDy53o,Islah,0.869861
504684,Check,Young Thug,spotify:track:1jk9c0gZI0wbn5GvAj7PBV,Barter 6,0.749550
1819388,DNA.,Kendrick Lamar,spotify:track:6HZILIRieu8S0iqY8kIKhj,DAMN.,0.929789
1553751,Hendrix,Wyclef Jean,spotify:track:5MZjzyQUEhrfDnnzGsLqnx,J'ouvert (Deluxe Edition),0.657147
1162099,"Me, Myself & I",G-Eazy,spotify:track:40YcuQysJ0KlGQTeGUosTC,When It's Dark Out,0.913337
2254455,Some Way,NAV,spotify:track:7zxRMhXxJMQCeDDg0rKAVo,NAV,0.788293
387873,Party,Chris Brown,spotify:track:1KhljCoJ2TgDEldnnfTHHc,Heartbreak on a Full Moon,0.802104


## Gladiator - Hanz Zimmer

In [27]:
recommend = Compare[Compare['track_uri']=='spotify:track:1eRuo0cdRSqJClg7G6ZjdT']

In [28]:
recommend[['track_name','artist_name','track_uri','album_name','Popularity_Rating_Normalized']]

,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
479757,Earth,Hans Zimmer,spotify:track:1eRuo0cdRSqJClg7G6ZjdT,Gladiator - Music From The Motion Picture,0.398553


#### Recommendation Analysis
<br>- The resulting recommendations for this song are very strong.
<br>- The recommender system has identified that the song Gladiator is in a motion picture movie by just using its audio features, when we look at the album names of the recommended songs we can see they are all from movies.
<br>- It has also recommended another song by Hanz Zimmer from the movie The Dark Night Rises.
<br>- It has returned the same artists multiple times which tells us Hanz Zimmer is similar to them, they are John Williams, Thomas Newman and James Horner who are all well known creators of songs in movies.

In [29]:
Song_Recommender_System(recommend)

[[ 479757]
 [ 862435]
 [1915337]
 [2147060]
 [ 628812]
 [1952836]
 [ 882542]
 [1944861]
 [1428966]
 [ 108400]
 [2225583]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
479757,Earth,Hans Zimmer,spotify:track:1eRuo0cdRSqJClg7G6ZjdT,Gladiator - Music From The Motion Picture,0.398553
862435,Rose,James Horner,spotify:track:2yEt5pA29CwKNXSplwnhgn,Titanic: Original Motion Picture Soundtrack - ...,0.453319
1915337,I Need You,James Newton Howard,spotify:track:6cD2cnMRgchHjuFvCDSOCh,The Hunger Games: Catching Fire,0.326417
2147060,Meeting Aragog,John Williams,spotify:track:7cwgBkWqS4Ze39uN9ULaQ0,Harry Potter and The Chamber of Secrets/ Origi...,0.387323
628812,The Scavenger,John Williams,spotify:track:2AJHEH475SPH3XckX6KyBI,Star Wars: The Force Awakens,0.409955
1952836,The Wardrobe - Score,Harry Gregson-Williams,spotify:track:6kG9QZqKmjBDnfvQn3LaNw,"The Chronicles of Narnia: The Lion, The Witch ...",0.426645
882542,The Letter That Never Came,Thomas Newman,spotify:track:32YVCuvm2iXGkNIDDmSX0j,Lemony Snicket's: A Series of Unfortunate Even...,0.399403
1944861,Necessary Evil,Hans Zimmer,spotify:track:6iYBQWnKkxrtikcws3uFC2,The Dark Knight Rises,0.355391
1428966,The North Mountain,Christophe Beck,spotify:track:4vmcA4Gia58BX51G9adnEG,Frozen,0.450576
108400,Extended Multiple Singing Bowls Meditation,Vidura Barrios,spotify:track:0N7EiH4VlscuXnQ1hUtAsJ,Tibetan Singing Bowls: Journey into the 7 Chak...,0.377377


## Secrets - OneRepublic

In [30]:
recommend = Compare[Compare['track_uri']=='spotify:track:4zweVv3Wa6XgexhoUOzkYs']

In [31]:
recommend[['track_name','artist_name','track_uri','album_name','Popularity_Rating_Normalized']]

,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
1448435,Secrets,OneRepublic,spotify:track:4zweVv3Wa6XgexhoUOzkYs,Waking Up,0.808396


#### Recommendation Analysis
<br>- Here we can see that the first recommendation for this song is actually another song by OneRepublic from the exact same album 'If I Lose Myself' using only audio features.
<br>- The recommender system recommended very similar artists to that of the pop band OneRepublic like Ed Sheeran and Cage The Elpehant.
<br>- It can be seen that the recommended songs are all within a similar Popularity Rating as Secrets.

In [32]:
Song_Recommender_System(recommend)

[[1448435]
 [1614440]
 [2252132]
 [1702729]
 [1431160]
 [1161096]
 [ 591549]
 [1854701]
 [1191258]
 [ 889644]
 [1414540]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
1448435,Secrets,OneRepublic,spotify:track:4zweVv3Wa6XgexhoUOzkYs,Waking Up,0.808396
1614440,If I Lose Myself,OneRepublic,spotify:track:5Zc3XI1rq7opdZUwFUdKCj,Native,0.692391
2252132,Light,San Holo,spotify:track:7zTx8ePYAmPFQuxP3xlXZn,Light (Remixes) - EP,0.760978
1702729,All Apologies,Nirvana,spotify:track:5sWMEcMemPH3Bc8hxeivIi,In Utero - 20th Anniversary Remaster,0.613720
1431160,Highway Don't Care,Tim McGraw,spotify:track:4wFUdSCer8bdQsrp1M90sa,Two Lanes Of Freedom,0.779780
1161096,Lips Of An Angel,Hinder,spotify:track:40LQiUUUKXVGyNs09lHVjW,Extreme Behavior,0.772947
591549,All Of The Lights,Kanye West,spotify:track:22L7bfCiAkJo5xGSQgmiIO,My Beautiful Dark Twisted Fantasy,0.838963
1854701,Castle on the Hill,Ed Sheeran,spotify:track:6PCUP3dWmTjcTtXY02oFdT,÷,0.861151
1191258,Runaway (U & I),Galantis,spotify:track:46lFttIf5hnUZMGvjK0Wxo,Pharmacy,0.858420
889644,You Make Me Brave (Live),Amanda Cook,spotify:track:345RHjiEdIVedsxtiwGG7t,You Make Me Brave (Live),0.711885


## Come Out Ye Black and Tans - The Fenians

In [33]:
recommend = Compare[Compare['track_uri']=='spotify:track:17hQnWxfJd1zlpcMu0nsrQ']

In [34]:
recommend[['track_name','artist_name','track_uri','album_name','Popularity_Rating_Normalized']]

,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
326798,Come Out Ye Black and Tans,The Fenians,spotify:track:17hQnWxfJd1zlpcMu0nsrQ,Have Fun Or Get Out!,0.097596


#### Recommendation Analysis
<br>- This was an expermental test to see how the recommender system would handle a less popular Irish song
<br>- The system has recognised what kind of song it is and has recommended the famous Irish song 'Whiskey in the Jar', this proves the stability of the recommender system.
<br>- All the other recommendations are all either live like The Fenians song or very upbeat and contain a guitar like the song

In [35]:
Song_Recommender_System(recommend)

[[ 326798]
 [1810211]
 [1090076]
 [ 518250]
 [2168678]
 [1890389]
 [1208558]
 [ 118987]
 [ 311646]
 [ 740799]
 [ 625872]]


,track_name,artist_name,track_uri,album_name,Popularity_Rating_Normalized
326798,Come Out Ye Black and Tans,The Fenians,spotify:track:17hQnWxfJd1zlpcMu0nsrQ,Have Fun Or Get Out!,0.097596
1810211,Não Vou Mais Chorar - Ao Vivo,João Neto & Frederico,spotify:track:6Fbk76HFGTzhUBcyAFLOUt,Só Modão,0.100671
1090076,Eclipse Oculto - Live In Brazil / 1998,Caetano Veloso,spotify:track:3l7uMrRLkWRG46OZHjqQJQ,Prenda Minha,0.076783
518250,Whiskey in the Jar,The High Kings,spotify:track:1meYWKdfmw0Veq0T8DIeyy,Four Friends Live,0.153329
2168678,Near Me,Yonder Mountain String Band,spotify:track:7hYXJU9FTSoKsgooee4tGh,"Mountain Tracks, Vol. 5",0.120674
1890389,Os Dias Vão - Ao Vivo,Maria Cecília & Rodolfo,spotify:track:6WonVkSP6xjlP1PyrvQwWO,Ao Vivo Em São Paulo,0.000000
1208558,El Borracho - En Vivo,Silvestre Dangond,spotify:track:4ASLN0Amo2RpmhJaN5P3RN,Mucho + Que Sigo Invicto,0.160295
118987,Revolt,Felin,spotify:track:0PNTnMtMO2Z3GCL5LczseY,Revolt,0.000000
311646,Cuyahoga,R.E.M.,spotify:track:14SLZ4LZyJIatjfV1AgKEw,Unplugged 1991/2001: The Complete Sessions,0.188995
740799,Traveling Shoes,The Clara Ward Singers,spotify:track:2YH99HwYAjbJUCrubIQykK,Meetin' Tonight!,0.025903


# Conclusion

After looking at all of these recommendations it can be said that the recommender system is strong and ready for implementation in a <b>Playlist Continuation System</b>